## Capstone Project

The next code is going to import some data abouth neighborhoods and localities from Bogota, Colombia. Aditionally, for each neighborhood is get the latitutde and longitude.

The final data, neighborhoods with latitude and longitude are then used in foursquare to extract the nearby venues. After the nearby venues are extraceted the idea is to cluster the data and find out what unobservable groups of neighborhoods there are in the city.

The goal is to find out what are the neighborhoods that are more approapiate for opening a new hotel. I hope to discover a cluster in which all the neighborhoods are good candidates for starting this project.

In [1]:
# Import the requiered libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
! pip install geocoder
import geocoder
from pandas import ExcelWriter
from pandas import ExcelFile

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import io

    100% |████████████████████████████████| 102kB 6.6MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

### Scraping and Wrangling data

In [5]:
url = 'https://github.com/pmsanchezb/Capstone_project_IBM/blob/master/localidades_bogota.csv'
html = urlopen(url)
bs = BeautifulSoup(html,"html.parser")

### Extract table

In [6]:
table_body=bs.find('tbody')
rows = table_body.find_all('tr')
pc_bogota = []
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    pc_bogota.append(cols)

Convert to dataframe

In [7]:
df_bogota = pd.DataFrame(pc_bogota)
df_bogota.columns = ['N','Locality', 'UPZ','Neighborhood']
df_bogota = df_bogota.drop('N', 1)

In [8]:
df_bogota.head()

,Locality,UPZ,Neighborhood
0,Usaquén,Paseo de los libertadores,Canaima
1,Usaquén,Paseo de los libertadores,La floresta de la sabana
2,Usaquén,Paseo de los libertadores,Torca
3,Usaquén,Verbenal,Balmoral norte
4,Usaquén,Verbenal,Buenavista


### Get latitude and longitude

In [9]:

# initialize your variable to None
df_coord = pd.DataFrame(index=df_bogota.index,columns=['Latitude','Longitude'])
j = 0
for neighs in df_bogota['Neighborhood']: 
    lat_lng_coords = None
    i = 0
    # loop until you get the coordinates
    while(lat_lng_coords is None)&(i<300):
        g = geocoder.arcgis('{}, Bogota, Colombia'.format(neighs))
        lat_lng_coords = g.latlng
        i=i+1
        if lat_lng_coords != None:
            latitude = lat_lng_coords[0]
            longitude = lat_lng_coords[1]
            df_coord.iloc[j] = lat_lng_coords
            print(lat_lng_coords)
    j = j + 1

[4.7740400000000704, -74.04108999999994]
[4.697340000000054, -74.07561999999996]
[4.808210000000031, -74.03010999999998]
[4.703899974464835, -74.04576998167808]
[4.766430000000071, -74.02589999999998]
[4.763280000000066, -74.02437999999995]
[4.766080000000045, -74.01597999999996]
[4.76144000000005, -74.03677999999996]
[4.762550000000033, -74.04010999999997]
[4.738140025535299, -74.07411998167812]
[4.761380000000031, -74.02193999999997]
[4.6128100000000245, -74.06670999999994]
[4.603530000000035, -74.07234999999997]
[4.763150033325538, -74.04384999610485]
[4.714299948929583, -74.1264299961048]
[4.610250000000065, -74.07285999999993]
[4.552190000000053, -74.15882999999997]
[4.763100000000065, -74.02334999999994]
[4.756880000000024, -74.02828999999997]
[4.758860000000027, -74.03740999999997]
[4.783330000000035, -74.03332999999998]
[4.62587000000002, -74.17981999999995]
[4.625020000000063, -74.19787999999994]
[4.627240000000029, -74.07864999999998]
[4.753030000000024, -74.02531999999997]
[

Join df_coord to df_bogota

In [12]:
df_bogota_coord = pd.concat([df_bogota, df_coord], axis=1)
df_bogota_coord.head()

,Locality,UPZ,Neighborhood,Latitude,Longitude
0,Usaquén,Paseo de los libertadores,Canaima,4.77404,-74.0411
1,Usaquén,Paseo de los libertadores,La floresta de la sabana,4.69734,-74.0756
2,Usaquén,Paseo de los libertadores,Torca,4.80821,-74.0301
3,Usaquén,Verbenal,Balmoral norte,4.7039,-74.0458
4,Usaquén,Verbenal,Buenavista,4.76643,-74.0259


In [13]:

address = 'Bogota'

geolocator = Nominatim(user_agent="bogota_explorer")
location = geolocator.geocode(address)
latitude_bogota = location.latitude
longitude_bogota = location.longitude
print('The geograpical coordinate of Bogota are {}, {}.'.format(latitude_bogota, longitude_bogota))

The geograpical coordinate of Bogota are 4.5980772, -74.0761028.


In [14]:
# Create map of Toronto using latitude and longitude values
map_bogota = folium.Map(location=[latitude_bogota, longitude_bogota], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_bogota_coord['Latitude'], df_bogota_coord['Longitude'], df_bogota_coord['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bogota)  
    
map_bogota

In [16]:
# The code was removed by Watson Studio for sharing.

### Explore neighborhoods in Bogotá

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
bogota_venues = getNearbyVenues(names=df_bogota_coord['Neighborhood'],
                                   latitudes=df_bogota_coord['Latitude'],
                                   longitudes=df_bogota_coord['Longitude']
                                  )

Canaima
La floresta de la sabana
Torca
Balmoral norte
Buenavista
Chaparral
El codito
El refugio de San Antonio
El verbenal
Horizontes
La estrellita
La frontera
La llanurita
Marantá
Maturín
Medellín
Mirador del norte
Nuevo horizonte
San antonio norte
Santandersito
Tibabita
Viña del mar
Bosque de San Antonio
El pite
El redil
La cita
La granja norte
La uribe
Los naranjos
San juan bosco
Urbanización los laureles
Ainsuca
Altablanca
Barrancas
California
Cerro norte
Danubio
La perla oriental
Milán
Pradera norte
San cristóbal norte
San cristóbal norte parte alta
San cristóbal norte parte baja
Santa teresa
Torcoroma
Villa nydia
Villa oliva
El toberín
Babilonia
Darandelos
Estrella del norte
Guanoa
Jardín norte
La liberia
La pradera norte
Las orquídeas
Santa mónica
Villa magdala
Villas de aranjuez
Villas del mediterraneo
Zaragoza
Acacias
Antigua
Belmira
Bosque de pinos
Caobos salazar
Capri
Cedritos
Cedro bolívar
Cedro golf
Cedro madeira
Cedro narvaez
Cedro salazar
El contador
El rincón de las mar

In [17]:
print(bogota_venues.shape)
bogota_venues.head()

(7534, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Canaima,4.77404,-74.04109,Pavo Munich,4.776968,-74.041793,Sandwich Place
1,Canaima,4.77404,-74.04109,Tugo Calle 197,4.775796,-74.042030,Furniture / Home Store
2,Canaima,4.77404,-74.04109,Club el Rancho,4.774056,-74.045122,Golf Course
3,Canaima,4.77404,-74.04109,Flores De La 200,4.776016,-74.041630,Flower Shop
4,Canaima,4.77404,-74.04109,Hamburguesas El Corral,4.771193,-74.042496,Burger Joint


In [18]:
# one hot encoding
bogota_onehot = pd.get_dummies(bogota_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bogota_onehot['Neighborhood'] = bogota_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bogota_onehot.columns[-1]] + list(bogota_onehot.columns[:-1])
bogota_onehot = bogota_onehot[fixed_columns]
bogota_onehot.head()

,Neighborhood,ATM,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
bogota_grouped = bogota_onehot.groupby('Neighborhood').mean().reset_index()
bogota_grouped

,Neighborhood,ATM,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Acacias,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
1,Acevedo tejada,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.052632,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
2,Ainsuca,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.014085,0.000000,0.0
3,Alfonso López norte,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.032258,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
4,Altablanca,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
5,Antigua,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.060606,0.000000,0.0
6,Antiguo country,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.015152,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
7,Armenia,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
8,Atahualpa,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
9,Babilonia,0.0,0.041667,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0


In [20]:
num_top_venues = 5

for hood in bogota_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bogota_grouped[bogota_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acacias----
                   venue  freq
0                 Lawyer   1.0
1              Nightclub   0.0
2  Outdoors & Recreation   0.0
3        Other Nightlife   0.0
4        Organic Grocery   0.0


----Acevedo tejada----
                venue  freq
0          Restaurant  0.21
1               Diner  0.11
2  Seafood Restaurant  0.05
3          Steakhouse  0.05
4  Chinese Restaurant  0.05


----Ainsuca----
                venue  freq
0          Restaurant  0.13
1  Italian Restaurant  0.06
2        Burger Joint  0.06
3                Café  0.04
4      Sandwich Place  0.04


----Alfonso López norte----
                       venue  freq
0                 Restaurant  0.10
1  South American Restaurant  0.10
2                        Bar  0.06
3       Gym / Fitness Center  0.06
4                Comedy Club  0.03


----Altablanca----
                  venue  freq
0          Skating Rink   0.1
1             BBQ Joint   0.1
2  Gym / Fitness Center   0.1
3                Bakery   0.1
4  Fast 

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bogota_grouped['Neighborhood']

for ind in np.arange(bogota_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bogota_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acacias,Lawyer,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Yoga Studio,Dumpling Restaurant
1,Acevedo tejada,Restaurant,Diner,Chinese Restaurant,Breakfast Spot,Seafood Restaurant,Steakhouse,Café,Latin American Restaurant,Juice Bar,Caribbean Restaurant
2,Ainsuca,Restaurant,Italian Restaurant,Burger Joint,Sandwich Place,Coffee Shop,Café,BBQ Joint,Bar,Tapas Restaurant,Building
3,Alfonso López norte,Restaurant,South American Restaurant,Bar,Gym / Fitness Center,Art Gallery,Café,BBQ Joint,Fried Chicken Joint,Juice Bar,Martial Arts Dojo
4,Altablanca,Donut Shop,Fast Food Restaurant,Farmers Market,Coffee Shop,Seafood Restaurant,BBQ Joint,Health & Beauty Service,Skating Rink,Gym / Fitness Center,Bakery


### Clustering Data

k = 10 clusters. I hope one cluster to have a high concentration of hotels.

In [61]:
# set number of clusters
kclusters = 15

bogota_grouped_clustering = bogota_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bogota_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([7, 3, 6, 6, 6, 6, 6, 6, 4, 6], dtype=int32)

In [62]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bogota_merged = df_bogota_coord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bogota_merged = bogota_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bogota_merged.head() 

,Locality,UPZ,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Usaquén,Paseo de los libertadores,Canaima,4.77404,-74.0411,6.0,Furniture / Home Store,Department Store,Burger Joint,Flower Shop,Soccer Field,BBQ Joint,Bus Station,Sandwich Place,Golf Course,Grocery Store
1,Usaquén,Paseo de los libertadores,La floresta de la sabana,4.69734,-74.0756,6.0,Pizza Place,South American Restaurant,Latin American Restaurant,Bakery,Deli / Bodega,Bar,BBQ Joint,Restaurant,Supermarket,Steakhouse
2,Usaquén,Paseo de los libertadores,Torca,4.80821,-74.0301,6.0,Athletics & Sports,Soccer Field,Restaurant,Farm,Latin American Restaurant,Farmers Market,Fast Food Restaurant,Falafel Restaurant,Exhibit,Drugstore
3,Usaquén,Verbenal,Balmoral norte,4.7039,-74.0458,6.0,Bakery,Clothing Store,Seafood Restaurant,Hotel,Café,Restaurant,Bed & Breakfast,Spa,Big Box Store,Brewery
4,Usaquén,Verbenal,Buenavista,4.76643,-74.0259,1.0,Brewery,Fast Food Restaurant,Lighthouse,Dog Run,Bus Station,Farmers Market,Farm,Falafel Restaurant,Exhibit,Eastern European Restaurant


### Analize clusters and identify the best locality/neighborhood for opnening a new hotel

In [ ]:
Before you continue the reading of the code. Our analysis of the clusters indicate us that cluster number 4 is the more appropiate because the neighborhoods that belong to it are in most of the cases 

#### Examining cluster 1

In [63]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 0, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Usaquén,Chaparral,0.0,Lighthouse,Yoga Studio,Exhibit,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Dumpling Restaurant


#### Examining cluster 2

In [64]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 1, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Usaquén,Buenavista,1.0,Brewery,Fast Food Restaurant,Lighthouse,Dog Run,Bus Station,Farmers Market,Farm,Falafel Restaurant,Exhibit,Eastern European Restaurant
7,Usaquén,El refugio de San Antonio,1.0,Bakery,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Yoga Studio
8,Usaquén,El verbenal,1.0,Gastropub,Bus Station,Pizza Place,Seafood Restaurant,Ice Cream Shop,Bakery,Grocery Store,Gym,Empanada Restaurant,Eastern European Restaurant
14,Usaquén,Maturín,1.0,Supermarket,Food Court,Fish & Chips Shop,Fast Food Restaurant,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service
16,Usaquén,Mirador del norte,1.0,Fried Chicken Joint,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Yoga Studio,Drugstore
19,Usaquén,Santandersito,1.0,Playground,Fast Food Restaurant,Recreation Center,Bakery,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
28,Usaquén,Los naranjos,1.0,Motorcycle Shop,Breakfast Spot,Shoe Store,Gift Shop,Fried Chicken Joint,Caribbean Restaurant,Bar,Pizza Place,Falafel Restaurant,Farm
30,Usaquén,Urbanización los laureles,1.0,Fried Chicken Joint,Costume Shop,Clothing Store,Yoga Studio,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
45,Usaquén,Villa nydia,1.0,Latin American Restaurant,Plaza,Airport,Fast Food Restaurant,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
53,Usaquén,La liberia,1.0,Bakery,Sandwich Place,Park,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


#### Examining cluster 3

In [65]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 2, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Usaquén,Viña del mar,2.0,Health & Beauty Service,Department Store,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Yoga Studio
37,Usaquén,La perla oriental,2.0,Coffee Shop,Convenience Store,Gym / Fitness Center,Mexican Restaurant,Bakery,Tailor Shop,Café,Juice Bar,Department Store,Donut Shop
59,Usaquén,Villas del mediterraneo,2.0,Donut Shop,Fruit & Vegetable Store,Convenience Store,Burger Joint,Café,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
212,Fontibón,Betania,2.0,Health & Beauty Service,Fast Food Restaurant,Department Store,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Yoga Studio
248,Fontibón,La aldea,2.0,Department Store,Convenience Store,Donut Shop,Soccer Stadium,Automotive Shop,Yoga Studio,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
259,Fontibón,Moravia,2.0,Burger Joint,Café,Coffee Shop,Yoga Studio,Event Space,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Exhibit
270,Fontibón,Visión semiindustrial,2.0,Donut Shop,Coffee Shop,Park,Department Store,Bus Stop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Dumpling Restaurant
283,Fontibón,Santa cecilia,2.0,Donut Shop,Burger Joint,Snack Place,Yoga Studio,Event Space,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Exhibit
284,Fontibón,Tarragona,2.0,Donut Shop,Café,Automotive Shop,Department Store,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space


#### Examining cluster 4

In [66]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 3, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Usaquén,Capri,3.0,Shopping Mall,Restaurant,Mobile Phone Shop,Food & Drink Shop,Mexican Restaurant,Park,Yoga Studio,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
83,Usaquén,Bella suiza,3.0,Restaurant,Fast Food Restaurant,Gym,Shop & Service,Market,Bakery,BBQ Joint,Cafeteria,German Restaurant,Department Store
84,Usaquén,Bellavista,3.0,Music Store,Pizza Place,Pharmacy,Check Cashing Service,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service
91,Usaquén,Ginebra,3.0,Spa,Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
93,Usaquén,La glorieta,3.0,Shopping Mall,Mobile Phone Shop,Food & Drink Shop,Park,Yoga Studio,Event Space,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service
94,Usaquén,Las delicias del Carmen,3.0,Mobile Phone Shop,Dessert Shop,Department Store,Restaurant,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
96,Usaquén,San gabriel,3.0,Sandwich Place,Bakery,Art Museum,Clothing Store,Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
98,Usaquén,Santa ana occidental,3.0,Hotel,Park,Café,Coffee Shop,Grocery Store,Pizza Place,Shopping Mall,Lounge,French Restaurant,Supermarket
99,Usaquén,Santa bárbara,3.0,Shopping Mall,Pizza Place,Pub,Multiplex,Shop & Service,Food & Drink Shop,Gym / Fitness Center,Restaurant,Bakery,Rental Car Location
100,Usaquén,Santa bárbara alta,3.0,Shopping Mall,Pizza Place,Pub,Multiplex,Shop & Service,Food & Drink Shop,Gym / Fitness Center,Restaurant,Bakery,Rental Car Location


#### Examining cluster 5

In [67]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 4, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Usaquén,Lisboa,4.0,Ice Cream Shop,Park,Event Service,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit
90,Usaquén,Francisco Miranda,4.0,Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Exhibit,Drugstore
127,Chapinero,El refugio,4.0,Construction & Landscaping,Moving Target,Park,Latin American Restaurant,Event Service,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
205,La candelaria,Belén,4.0,Food & Drink Shop,Latin American Restaurant,Bar,Park,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
208,Fontibón,Atahualpa,4.0,Construction & Landscaping,Moving Target,Park,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Exhibit
211,Fontibón,Belén,4.0,Food & Drink Shop,Latin American Restaurant,Bar,Park,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
218,Fontibón,El jordan,4.0,Park,Seafood Restaurant,Fast Food Restaurant,BBQ Joint,Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
222,Fontibón,Ferrocaja,4.0,Fried Chicken Joint,Park,Pharmacy,Yoga Studio,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
227,Fontibón,La laguna,4.0,ATM,Park,Nature Preserve,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Drugstore,Event Space
244,Fontibón,El refugio,4.0,Construction & Landscaping,Moving Target,Park,Latin American Restaurant,Event Service,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


#### Examining cluster 6

In [68]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 5, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Usaquén,Bosque de San Antonio,5.0,Convenience Store,Yoga Studio,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
236,Fontibón,Valle verde,5.0,Convenience Store,Yoga Studio,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
302,Antonio Nariño,San Antonio,5.0,Convenience Store,Yoga Studio,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant


#### examining cluster 7

In [69]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 6, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Usaquén,Canaima,6.0,Furniture / Home Store,Department Store,Burger Joint,Flower Shop,Soccer Field,BBQ Joint,Bus Station,Sandwich Place,Golf Course,Grocery Store
1,Usaquén,La floresta de la sabana,6.0,Pizza Place,South American Restaurant,Latin American Restaurant,Bakery,Deli / Bodega,Bar,BBQ Joint,Restaurant,Supermarket,Steakhouse
2,Usaquén,Torca,6.0,Athletics & Sports,Soccer Field,Restaurant,Farm,Latin American Restaurant,Farmers Market,Fast Food Restaurant,Falafel Restaurant,Exhibit,Drugstore
3,Usaquén,Balmoral norte,6.0,Bakery,Clothing Store,Seafood Restaurant,Hotel,Café,Restaurant,Bed & Breakfast,Spa,Big Box Store,Brewery
9,Usaquén,Horizontes,6.0,Café,Food Truck,Burger Joint,Tennis Stadium,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
11,Usaquén,La frontera,6.0,Restaurant,Italian Restaurant,Coffee Shop,Café,Peruvian Restaurant,Mexican Restaurant,Hotel,Tapas Restaurant,Bar,Spanish Restaurant
12,Usaquén,La llanurita,6.0,Café,Restaurant,BBQ Joint,South American Restaurant,Seafood Restaurant,Italian Restaurant,Burger Joint,Electronics Store,Supermarket,Snack Place
13,Usaquén,Marantá,6.0,Coffee Shop,Italian Restaurant,Clothing Store,Café,Bakery,Latin American Restaurant,Burger Joint,Bookstore,Ice Cream Shop,Mexican Restaurant
15,Usaquén,Medellín,6.0,Theater,Burger Joint,Mexican Restaurant,Movie Theater,Café,Restaurant,Sandwich Place,Caribbean Restaurant,Nightclub,Art Museum
18,Usaquén,San antonio norte,6.0,Market,Seafood Restaurant,Food & Drink Shop,Soccer Field,Music Venue,Yoga Studio,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


#### Examining cluster 8

In [70]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 7, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Usaquén,Acacias,7.0,Lawyer,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Yoga Studio,Dumpling Restaurant
145,Chapinero,Las acacias,7.0,Lawyer,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Yoga Studio,Dumpling Restaurant


#### Explorting cluster 9

In [71]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 8, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Usaquén,Bosque de pinos,8.0,Pub,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Drugstore


#### Exploring cluster 10

In [72]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 9, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
210,Fontibón,Santiago Batavia,9.0,Café,Grocery Store,Yoga Studio,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
223,Fontibón,Flandes,9.0,Café,Grocery Store,Park,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service


#### Explorting cluster 11

In [73]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 10, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
245,Fontibón,El triángulo,10.0,Theater,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Exhibit,Donut Shop


#### Exploring cluster 12

In [74]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 11, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
74,Usaquén,El rincón de las margaritas,11.0,Construction & Landscaping,Shopping Mall,Grocery Store,Restaurant,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
76,Usaquén,Las margaritas,11.0,Construction & Landscaping,Shopping Mall,Grocery Store,Restaurant,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
131,Chapinero,Toscana,11.0,Construction & Landscaping,Athletics & Sports,Mexican Restaurant,Exhibit,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
172,Teusaquillo,San luis,11.0,Smoke Shop,Construction & Landscaping,Furniture / Home Store,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Yoga Studio
238,Fontibón,Versalles,11.0,Construction & Landscaping,Yoga Studio,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
